In [2]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools, ConfluentTableDescriptor
from pyflink.table import TableEnvironment, DataTypes, Schema
from pyflink.table.expressions import *
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.window import Tumble, Slide, Session
from pyflink.table.udf import udf

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
print(os.getenv("CLOUD_REGION"))

us-west1


In [5]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

It is important to have the same schema between Flink and Kafka Topic.

In [6]:
env.use_catalog("msds682")
env.use_database("msds682")
env.execute_sql("""DROP TABLE IF EXISTS `system.usage.day12`""")

In [7]:
env.execute_sql("""
    CREATE TABLE IF NOT EXISTS `system.usage.day12` (
        id STRING,
        cpu_usage FLOAT,
        cpu_stats ARRAY<FLOAT>,
        memory_usage FLOAT,
        `timestamp` TIMESTAMP_LTZ(3),
        WATERMARK FOR `timestamp` AS `timestamp` - INTERVAL '5' SECOND
    ) WITH (
        'value.format'='avro-registry',
        'connector' = 'confluent'
    )
""")

In [8]:
table = env.from_path("`system.usage.day12`")

In [21]:
windowed_table = table.window(Tumble.over(lit(4).second)
            .on(col("timestamp"))
            .alias("w")           
)
result = windowed_table.group_by(
    col("w"), 
    col("id")
).select(
    col("id"),
    col("memory_usage").min.alias("min_memory_usage"),
    col("memory_usage").max.alias("max_memory_usage"), # Access 'max' as a property
    col("memory_usage").count.alias("record_ct"),
    col("w").start.alias("window_start"),              
    col("w").end.alias("window_end"),
    col("timestamp").min.alias("min_processing_time"),
    col("timestamp").max.alias("max_processing_time")               
)

In [ ]:
ConfluentTools.print_changelog(result)

| +I |                              3 |                           74.0 |                           74.0 |                    1 | 2025-11-24 19:40:00.000 | 2025-11-24 19:40:04.000 | 2025-11-24 11:40:05.206 | 2025-11-24 11:40:05.206 |
| +I |                              3 |                           74.0 |                           74.0 |                    1 | 2025-11-24 19:40:00.000 | 2025-11-24 19:40:03.000 |
| +I |                              3 |                           74.0 |                           74.0 |                    1 | 2025-11-24 19:40:00.000 | 2025-11-24 19:40:03.000 |
| +I |                              3 |                           74.0 |                           74.0 |                    1 | 2025-11-24 19:40:00.000 | 2025-11-24 19:40:04.000 |
| +I |                              3 |                           74.0 |                           74.0 |                    1 | 2025-11-24 19:40:00.000 | 2025-11-24 19:40:03.000 |
| +I |                              3 |    